In [ ]:
import numpy as np
import pandas
from minigo import dao
from glob import glob
#[sorted(choice), 1 - drop, drop, meandrop, a, fitloss]

def endValueDiff(s):
    slope, b = np.polyfit(np.arange(len(s)), s, 1)
    return s[-1] / ((len(s) - 1) * slope + b) - 1

def fitLossAbsMax(s):
    slope, b = np.polyfit(np.arange(len(s)), s, 1)
    line = np.arange(len(s)) * slope + b
    return np.abs(s / line - 1).max()
    
ret = {}
for npy in glob('*_n.npy'):
    day = npy.replace('_n.npy', '')
    if day == '2008-08-04':
        x = pandas.DataFrame(np.load('lasttime.npy', allow_pickle=True))
    else:
        x = pandas.DataFrame(np.load(npy, allow_pickle=True))
    df = pandas.read_csv(day + '_d.csv', index_col=0)
    print(day)
    otherMetrics = []
    for a in x.values:
        s = dao.get_dpr(df.iloc[a[0]])[0].sum(axis=0) / len(a[0])
        otherMetrics.append([endValueDiff(s), fitLossAbsMax(s)])

    otherMetrics = pandas.DataFrame(otherMetrics)
    otherMetrics.index = x.index
    otherMetrics.columns = ['endValueDiff', 'fitLossAbsMax']
    x = pandas.concat([x, otherMetrics], axis=1)

    x = x[x['fitLossAbsMax'] < 0.01].sort_values(4).iloc[-1]
    print(x['fitLossAbsMax'], x[4] * df.shape[1])
    
    stocks = df.index[x[0]].tolist()
    dff = df.iloc[x[0]]
    d, p, _, _ = dao.get_dpr(dff)
    ret[day] = [list(map(lambda a: '@"' + a.split(' ')[1][1:-2] + '"@', stocks)), (p / p.sum()).tolist(), x[0], x[2], x[4]]

In [ ]:
str(ret).replace('\'@', '').replace('@\'', '').replace('\'', '"')

In [ ]:
def demo(day):
    df = pandas.read_csv(day + '_d.csv', index_col=0)
    d,p,_,_=dao.get_dpr(df.iloc[ret[day][2]])
    s = d.sum(axis=0) / 10
    stat = dao.stat(s)
    print(day, stat[0], stat[2] * len(s) / s[0] * 100, stat[3])
    _, b = np.polyfit(np.arange(s.shape[0]), s, 1)
    s2 = np.arange(len(s)) * stat[2] + b
    pandas.DataFrame([s,s2]).T.plot(title=day)

for x in ret:
    demo(x)

In [ ]:
import os
from minigo import dao 
dao.init('2008-08-04_d.csv')
if os.path.exists("lasttime.npy"):
    os.remove("lasttime.npy") 
os.system('rm -rf abc;mkdir abc')
dao.playByRandom('abc', 100)

In [ ]:
for i in range(10):
    dao.playByRandom('abc', 100)

In [ ]:
import pandas
import numpy as np
x = pandas.DataFrame(np.load('lasttime.npy', allow_pickle=True))
df = pandas.read_csv('2008-08-04_d.csv', index_col=0)
x[4] = x[4] * df.shape[1] * 10
x = x[x[5]<0.01].sort_values(4);
x